# Instrucciones.

En el laboratorio realice las siguientes tareas

* Complete la celda "A. Datos grupo".
* Descargue los archivos de instancias de ucursos.
* Complete la sección "B. preparación"
* Lea el enunciado y complete los ejercicios 1, 2 y 3.
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 2 presencial 
* Plazo de entrega:  Miércoles 6 de mayo las 18:00

La parte no presencial del laboratorio corresponde a los ejercicios 4, 5 y 6.
Se recomienda realizar esta parte durante el laboratorio, pero tiene tiempo de entrega hasta el día Viernes 8 de mayo a las 22:00

* Complete todos los ejercicios
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 2 completo
* Plazo de entrega: Viernes 8 de mayo a las 22:00 
  (puede solicitar por correo electrónico plazo adicional si lo necesita, pero su solicitud debe hacerse antes del Viernes 8 a    las 22:00. No es automático)


# A. Datos grupo

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: XXX

- Integrante 2: YYY

# B. Preparación

* Usaremos los paquetes JuMP, Gurobi, Distances y Plots.
  Descomente las lineas si no tiene los paquetes y asegúrese de tener la última versión de JuMP corriendo (0.21.2 en mayo 2020)

In [ ]:
import Pkg
#Pkg.add("Distances")
#Pkg.add("Plots")
#Pkg.update("JuMP")
Pkg.status("JuMP")

In [ ]:
using JuMP, Gurobi, Distances, Plots
const GUROBI_ENV = Gurobi.Env() #Esta referencia  nos servirá para usar solo un ambiente de Gurobi.

* En este problema calcularemos tours que pasan por ciudades en el plano, para cargar los archivos de entrada y visualizar estos tours usaremos algunas funciones pre-escritas. El archivo "preparacion.jl", escrito a continuación, tiene las funciones que necesitaremos.

```julia
dibuja(coordx,coordy,arcos)
## Recibe dos arreglos de N valores donde (coordx[i],coordy[i]) son las coordenadas de la ciudad i
## Recibe además una matriz arcos de N x N, donde arcos[i,j] es el peso del arco [i,j]
## Dibuja los N puntos en el plano y dibuja los arcos con ancho de linea proporcional al peso.

lee_archivo(nombre_archivo)
## Recibe un archivo con las coordenadas de N ciudades, devolviendo N, las coordenadas x, las coordenadas y.

encuentra_ciclo(arcos)
#Si arcos es una matriz 0-1 (o cercana a serlo) tal que el nodo 1 está en un único ciclo, esta función lo reporta.
```

* Cargue el archivo usando el siguiente comando.

In [ ]:
include("preparacion.jl");

* Pruebe las funciones auxiliares cargando el archivo "ejemplo.txt" (con 25 puntos), y dandole peso a algunas aristas como sigue:

In [ ]:
N,x_pos,y_pos=lee_archivo("ejemplo.txt")
aristas_ejemplo=zeros(N,N)
aristas_ejemplo[11,14]=1
aristas_ejemplo[1,20]=0.1
aristas_ejemplo[18,23]=0.5
dibuja(x_pos,y_pos,aristas_ejemplo)

In [ ]:
ciclo_ejemplo=zeros(N,N)
ciclo_ejemplo[1,7]=1
ciclo_ejemplo[7,23]=1
ciclo_ejemplo[23,5]=1
ciclo_ejemplo[5,1]=1
dibuja(x_pos,y_pos,ciclo_ejemplo)
encuentra_ciclo(ciclo_ejemplo)

# C. MTZ para el problema del vendedor viajero.

En el problema del vendedor viajero (TSP), recibimos N ciudades del plano y deseamos encontrar un tour (ciclo) que pase por cada punto exactamente una vez, de largo total mínimo. Este problema es NP-completo y difícil de abordar en toda generalidad.
En este laboratorio describiremos un programa entero de tamaño exponencial (en N) para resolver TSP, y luego usaremos técnicas de generación de filas para resolver este problema.

Hay *varias* formas de modelar TSP como PLE, algunas de ellas tienen tamaño polinomial. Para esto supondremos que $d(i,j)$ es la distancia del punto $i$ al punto $j$ y $z_{i,j}$ es la indicatriz del arco $(i,j)$ en el tour.

Por ejemplo, el siguiente modelo conocido como MTZ (Miller, Tucker, Zemlin) tiene "solo" $O(N^2)$ variables y restricciones.
(La variable $u_i$ representa la manera el orden en el cual son visitados los nodos).

\begin{align*}
(\text{MTZ})\qquad \min \sum_{i=1}^N\sum_{j=1}^N z_{i,j}d(i,j)&\\
z_{i,i}&=0,&\forall i\in [N]\\
\sum_{j=1}^N z_{i,j} &= 1,& \forall i\in [N]\\
\sum_{j=1}^N z_{j,i} &= 1,&\forall i\in [N]\\
u_i−u_j+(N−1)z_{i,j}&\leq N−2, &\forall i,j \in [N], i\geq 2, j\geq 2\\
z &\in \{0,1\}^{[N]\times [N]}\\
u&\in \mathbb{R}^{[N]\setminus \{1\}}
\end{align*}

Sin embargo, en la práctica no es muy efectivo. Para 25 ciudades ya toma algunos segundos como se ve a continuación.
Para grandes cantidades de ciudades se vuelve muy lento pues la formulación está muy lejos de ser exacta.

Al principio del laboratorio probemos el modelo MTZ (nos servirá para recordar como se escribe un PLM)

In [ ]:
function resuelveMTZ(nombre_archivo)
## Resuelve MTZ, devuelve el valor, el ciclo, y dibuja la solución
  
    #Cargar datos del archivo
    N,x_pos,y_pos = lee_archivo(nombre_archivo)

    #Crear un modelo nuevo. 
    #la variable GUROBI_ENV creada en el archivo preparación nos permite reutilizar el ambiente de Gurobi
    #en vez de crear uno nuevo cada vez que creamos un modelo (en particular, ya no saldrán múltiples
    #mensajes sobre la licencia académica)
    #(esta sintaxis se usa a partir de Jump 0.21)
    MTZ = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 1))

    #Crear variables, z es binaria, u es real.
    @variable(MTZ, z[1:N,1:N], Bin)
    @variable(MTZ, u[2:N])

    #Crear objetivo (usamos "euclidean" del paquete Distances para calcular la distancia euclideana)
    @objective(MTZ, Min, sum(z[i,j]*euclidean([x_pos[i],y_pos[i]],[x_pos[j],y_pos[j]]) for i=1:N,j=1:N))

    #Crear restricciones
    @constraints(MTZ, begin
        self[i=1:N], z[i,i]==0
        salida[i=1:N], sum(z[i,1:N])==1
        entrada[i=1:N], sum(z[1:N,i])==1
        prioridad[i=2:N, j=2:N], u[i]-u[j]+N*z[i,j]<=N-1
    end)
    
    #optimizar! (el modelo es siempre factible)
    optimize!(MTZ)
    
    #dibuja y retorna
    valor =  objective_value(MTZ)
    vector = value.(z) 
    ciclo = encuentra_ciclo(vector)
    dibuja(x_pos,y_pos,vector)
    return valor, ciclo
end

In [ ]:
@time valor,ciclo=resuelveMTZ("ejemplo.txt")
println("Valor= ",valor)
println("Ciclo= ",ciclo)

# D. DFZ para el problema del vendedor viajero.

## D1. Implementación mediante generación de filas y resolución iterativa de PLE.

En este laboratorio estudiaremos otro modelo para TSP mucho más efectivo, pero con una cantidad exponencial de restricciones.
El siguiente es el modelo DFZ (Dantzig-Fulkerson-Johnson)

\begin{align*}
(\text{DFZ})\qquad \min &\sum_{i=1}^N\sum_{j=1}^N z_{i,j}d(i,j)\\
z_{i,i}&=0,& \forall i\in [N]\\
\sum_{j=1}^N z_{i,j} &= 1,& \forall i\in [N]\\
\sum_{j=1}^N z_{j,i} &= 1,& \forall i\in [N]\\
z_{i,j}+z_{j,i}&\leq 1,& \forall i,j\in[N]\\
z(\delta^+(S)) &\geq 1,& \forall S\subseteq [N], S\neq N, 1\in S. (\text{corte}(S))\\
z &\in \{0,1\}^{[N]\times [N]}
\end{align*}


El modelo DFZ posee una cantidad exponencial de restricciones de tipo (corte). Esto hace intractable escribir todas las restricciones y luego usar un solver para resolverlo. En lugar de eso tomaremos la siguiente ruta.

1. Generar modelo con variables **integrales** sin restricción de corte y resolverlo.
   Notar que si $z$ es el vector solución entonces $z$ es unión de ciclos. 
2. Mientras $z$ tenga más de un ciclo, buscar un ciclo $C$ y llamar $S=V(C)$, notar que la restricción $z(\delta^+(S))\geq 1$ no está satisfecha y luego podemos **agregar al modelo la restricción corte($S$) y volver a resolver**
3. Cuando $z$ tenga un solo ciclo, ésta será la solución óptima.

## Ejercicio 1

Cree el modelo DFZ en Gurobi **sin incluir las restricciones de corte** completando la siguiente celda. 

Luego resuélvalo y dibuje la salida usando la función dibujar. 

In [ ]:
#Cargar nuevamente los datos de ejemplo (esto es para fijar los nombres de las variable)
N,x_pos,y_pos=lee_archivo("ejemplo.txt")
#Modelo DFZ
DFZ = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0)) #sin salida esta vez

#cree variables, objetivo y restricciones
@variable(..
@objective(..
@constraints(..

In [ ]:
#complete los comandos para optimizar y dibujar
optimize!(..
dibuja(..

## Ejercicio 2 

La función *encuentra_ciclo* (del archivo preparacion.jl) detecta el único ciclo que pasa por la ciudad 1 en un grafo que es unión de ciclos. Ejecute la función encuentra_ciclo en la matriz value.(z) con los valores encontrados por DFZ. Luego agregue a DFZ la restricción asociada al corte encontrado. Optimize y dibuje su nueva solución.

In [ ]:
# Ejecutar función
ciclo=encuentra_ciclo(..

#agregar nueva fila (restricción) a DFZ 
#(indicación: setdiff(1:N, H) devuelve la diferencia entre el conjunto [N] y los elementos de un arreglo H)
@constraint(DFZ,.. 

#optimize nuevamente y dibuje
optimize!(..
dibujar(..

## Ejercicio 3

En este ejercicio debe automatizar la generación de filas. Para esto

### 3.1) 
Complete primero la siguiente plantilla

In [ ]:
function resuelveDFZ(nombre_archivo)
    # recuperar N y puntos del archivo 
    ..
    
    # Crear modelo DFZ inicial 
    DFZ = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0)) #sin salida esta vez
    ..
    ..
    ..
    
    itera=1
    while (true) 
        
        ## optimizar
        optimize!(DFZ)
        
        ## encontrar el ciclo que contiene a 1
        ciclo = ..
        
        ## si el ciclo pasa por las N ciudades, calcular el valor del tour, escribir el ciclo y dibujarlo.
        if ..
            valorTour=..
            display(string("Número de iteraciones: ", itera))
            display(string("Tour óptimo encontrado, de largo ", valorTour))
            display(string("Lista de ciudades :", ciclo))
            dibuja(..
            return nothing
        else
            ## añadir a DFZ restricción de corte nuevo
            @constraint(..
        end     
    itera=itera+1
    end  
end

### 3.2) 
Ejecute su funcion en tres instancias (i30.txt, i40.txt, i50.txt), encontrando los tours óptimos

In [ ]:
resuelveDFZ(..

In [ ]:
resuelveDFZ(..

In [ ]:
resuelveDFZ(..

## D2. Implementación mediante generación de filas usando lazy-constraints-callbacks.

La técnica anterior permite resolver TSP para instancias de tamaño moderado resolviendo una serie de PLE
con más y más restricciones. Notamos que en cada uno de estos PLE, el solver debe realizar branch and bound 
(o más precisamente branch and cut), sin necesariamente reutilizar el trabajo anterior.

Una manera de hacer este proceso de manera más eficiente es realizar una sola ejecución de branch and cut 
y agregar estas restricciones al modelo no al final del mismo si no cada vez que se encuentra un incumbente.
Los solvers modernos permiten pausar branch and bound en algunos puntos clave, luego llamar a una función externa
y finalmente despausar la ejecución (a esto se le llama un callback).

En esta sección implementaremos una función callback del tipo lazy-constraint.
Básicamente una lazy-constraint es una restricción que no se revisa desde el principio de la ejecución si no que solo
es agregada al modelo una vez que encontramos un incumbente que no la satisface.

Es posible darle al solver una lista de lazy-constraints desde el principio o generarla via callbacks.

El modo de implementar una función callback en jump+gurobi (en genérico) es como sigue:
```julia
    #se crea un modelo (para este ejemplo, supongamos que tiene una variable x
    model = Model(... 
    @variable(model, x, ...
    @objective(
    @constraints(

    #esta función se llamará cuando Gurobi piensa que tiene una solución entera incumbente
    function mi_callback(cb_data)
        #comando para recuperar valor de la variable x y guardarlo en una variable temporal
        x_val = callback_value(cb_data, x)

        #Nota: la implementación actual de Gurobi podría llamar a esta función cuando la solución es fraccional por lo
        #      que conviene revisar integralidad.
        
        tolerancia=0.001
        if !(abs(x-round(x))<tolerancia)
          display(string("callback en solución fraccional"))
          return nothing
        end
        
        #sabemos que x es integral
        # buscar restriccion a agregar
        ..
        restriccion_nueva = @build_constraint( .. restriccion ..)
        MOI.submit(model, MOI.LazyConstraint(cb_data), restriccion)
        end
    end

    #mandar función callback al solver.
    MOI.set(model, MOI.LazyConstraintCallback(), mi_callback)

    #si optimizamos, el solver usará la función callback para confirmar factibilidad de sus soluciones, agregando
    #restricciones si es necesario
```

## Ejercicio 4. 

### 4.1) 
Complete la siguiente función para resolver DFZ usando callbacks.


In [ ]:
function resuelveDFZconCallbacks(nombre_archivo)
    # recuperar N y puntos del archivo 
    ..
    
    # Crear modelo DFZCall inicial 
    DFZCall = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0)) 
    @variable(..  
    @objective(..
    @constraints(..
                
    #implementar función callback
                
    function mi_callback(cb_data)
        display(string("Ejecutando callback"))
 
        ##Recupera variables z de cb_data
        z_val=zeros(N,N)
        for i=1:N, j=1:N
            z_val[i,j] = callback_value(cb_data, z[i,j])
        end    
        
        ##Revisar fraccionalidad
        tolerancia=0.001
        if (!all( x-> abs(x-round(x))<tolerancia, z_val))
          display(string("callback en solución fraccional"))
          return nothing
        end
        
        ## encontrar el ciclo que contiene a 1
        ciclo = ...
        
        if (...) ## si el ciclo no pasa por las N ciudades
            display(string("Agregando corte: ", ciclo))
            restriccion_nueva = @build_constraint(..
            MOI.submit(...
        end
    end                        
    
    MOI.set(...

    #Optimizar, reportar solución y dibujar.
    optimize!(DFZCall)
    valorTour =..
    ciclo =..
    display(string("Tour óptimo encontrado, de largo ", valorTour))
    display(string("Lista de ciudades :", ciclo))
    dibuja(..
    return valorTour, ciclo
end    

### 4.2) 
Ejecute su funcion en cuatro instancias (ejemplo.txt, i30.txt, i40.txt, i50.txt), encontrando los tours óptimos.

In [ ]:
resuelveDFZconCallbacks(..

In [ ]:
resuelveDFZconCallbacks(..

In [ ]:
resuelveDFZconCallbacks(..

In [ ]:
resuelveDFZconCallbacks(..

# E. Resolución de la relajación fraccional de DFZ

En instancias más grandes se hace complejo usar el esquema anterior pues estamos resolviendo demasiados PLE.
Una forma de mejorar esto es incluir (algunas) restricciones de corte no en el PLE, sino en su relajación fraccional.

\begin{align*}
(\text{DFZ-frac})\qquad \min &\sum_{i=1}^N\sum_{j=1}^N z_{i,j}d(i,j)\\
z_{i,i}&=0,& \forall i\in [N]\\
\sum_{j=1}^N z_{i,j} &= 1,& \forall i\in [N]\\
\sum_{j=1}^N z_{j,i} &= 1,& \forall i\in [N]\\
z_{i,j}+z_{j,i}&\leq 1,& \forall i,j\in[N]\\
z(\delta^+(S)) &\geq 1,& \forall S\subseteq [N], S\neq N, 1\in S. (\text{corte}(S))\\
z_{i,j}&\geq0,& \forall (i,j) \in [N]\times [N]
\end{align*}

El objetivo de esta sección es programar un método que resuelva la relajación fraccional de DFZ (no la integral).

Para esto, seguiremos un esquema similar al de la sección D.1. 

1. Generar modelo con variables **fraccionales** sin restricción de corte y resolverlo. 
2. Encontrar conjunto $S$ que contenga a 1 y que tenga corte $z(\delta^+(S))$ mínimo. Si este corte es de valor al menos 1 entonces $z$ será la solución óptima.
3. De otra forma, agregar la restricción asociada al corte de $S$ al modelo.

Para implementar el paso 2, lo ideal es usar un algoritmo combinatorial rápido para mincut como Stoer-Watson o Queyranne; o uno aleatorizado como Karger. En vez de eso, en este laboratorio calcularemos mincut mediante (una secuencia de) programas lineales puros. Básicamente calcularemos el $1$-$t$ corte mínimo para todo $t\neq 1$, y si alguno tiene valor menor que 1, agregaremos la restricción al modelo.

Usaremos el siguiente PL puro (vimos uno similar en auxiliar). 


\begin{align*}
min \sum_{i=1}^N\sum_{j=1}^N &q_{i,j}z_{i,j}\\
p_1&\geq 1\\
p_t&=0\\
p_i-p_j+q(j,i)&\geq 0\\
q&\ge 0
\end{align*}


Aquí las variables son los vectores q (largo) y p (potencial); el vector $z$ es dato. Básicamente el PL busca asignar potenciales entre 0 y 1 a cada vértice y llama $q(i,j)$ a la diferencia de potencial entre $i$ y $j$. En un mundo ideal, una solución óptima asignaría potenciales 0 a aquellos puntos en el lado de $t$ y potencial 1 a aquellos puntos en el lado de $1$ del corte y luego $q_e=1$ solo para los arcos que cruzan el corte. Esta idea también funciona de manera fraccional y de hecho se puede probar que 
$$S=\{i \in [N], p_i\geq \alpha\}$$
es un $1$-$ t$ corte minimo para cualquier $\alpha$ entre 0 y 1.

## Ejercicio 5. 

Implemente una funcion que reciba N, un indice t en 2:N, y un vector de pesos w (que toma el rol del z en el PL anterior) 
y devuelve el valor del minimo 1-t corte (usando pesos w) y un arreglo con los nodos al lado 1 de ese corte. 
Para esto complete la siguiente plantilla

In [ ]:
function cut(N, t, w)
    #calcula el mincut de 1 a t usando los valores de w
    mincut = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0)) #sin salida esta vez
    @variable(..
    @variable(..
    @objective(..
    @constraints(..
    optimize!(mincut)
    valor=...
    corte=... #Indicación: Use el comando findall para encontrar los i con p_i>α
              #(tipee ?findall en julia para ver como se usa)
    return valor, corte
end

Pruebe su funcion en el grafo con 6 vertices y pesos dados por el vector w siguiente (para t=2, 3, 6 debería obtener cortes de valor 1, 2 y 3 respectivamente)

In [ ]:
xxpos=[1, 1, 2, 4, 4, 5]
yypos=[1, 5, 3, 1, 5, 3]
w=zeros(6,6)
for (i,j,k) in [(1, 2, 1), (1, 3, 1), (2, 3, 1), (3, 4, 2), (3, 5, 2), (4, 6, 1), (5, 6, 3), (3,6,1), (2,5,1), (1,4,1)]
    w[i,j]=k
end

#dibujo
dibuja(xxpos,yypos,w)

#encontrando 1-t cortes para t=2,3,6
println(cut(6,2,w))
println(cut(6,3,w))
println(cut(6,6,w))

## Ejercicio 6

### 6.1) 
Implemente la siguiente función para calcular la solucion optima fraccional de DFZ similar a como lo hizo en el Ejercicio 3 (esta vez la solución no es necesariamente un ciclo)


In [ ]:
function resuelveDFZfraccional(nombrearchivo)
    # recuperar N y puntos del archivo 
    ..
    
    # Crear modelo DFZfrac inicial 
    DFZfrac = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0)) #sin salida esta vez

    @variable(..
    @objective(..
    @constraints(..
    
    iteracion=0
    listo=false
    while (!listo)
        listo=true #si no encontramos cortes nuevo en la ejecución, terminamos al final del loop.
        iteracion=iteracion+1
        
        ## optimizar y reportar valor.
        optimize!(DFZfrac)
        display(string("Iteración: ",iteracion, ", valor fraccional: ", objective_value(DFZfrac)))
        
        ## Encontrar algun 1-t corte de valor <1-epsilon, para epsilon=0.0001
        epsilon=0.0001
        for t in 2:N
            valor, corte = ..
            if (valor < 1-epsilon)
                @constraint(..
                display(string("Agregando corte: ", corte))
                listo=false
                break
            end
        end
    end
    
    valorTour=objective_value(DFZfrac)
    display(string("Tour fraccional encontrado de largo ", valorTour))
    dibuja(..
    return valorTour
end



### 6.2)
Ejecute su funcion en cuatro instancias (ejemplo.txt, i30.txt, i40.txt, i50.txt)), encontrando los tours fraccionales óptimos.

(Para referencia: el tour fraccional óptimo de "ejemplo.txt" tiene un valor cercano a 105.415)

In [ ]:
resuelveDFZfraccional(

In [ ]:
resuelveDFZfraccional(

In [ ]:
resuelveDFZfraccional(

In [ ]:
resuelveDFZfraccional(

# Comentario Final

Es posible interactuar con el solver mediante callbacks nuevamente para que en cada nodo de BnB, el solver también agregue cortes generados por el usuario (como los encontrados mediante mincut). En la actualidad, los algoritmos más veloces para TSP realizan esto.